In [1]:
import pandas as pd
import random
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from pyvis.network import Network
import community.community_louvain as community_louvain
from tqdm import tqdm

---

In [ ]:
# TODO : 
# Faire fonction de vérification des paires id: community pour le dataframe
# Revoir la méthode de sauvegarde des graphes (qui des anciens graphs)
# 

## Importation des données

In [2]:
project = pd.read_csv("../Data/CSV/Projects/project.csv", sep=";")

In [3]:
organization = pd.read_csv("../Data/CSV/Projects/organization.csv", sep=";")

On importera les pubications après

In [4]:
print(project.shape)
print(organization.shape)

(35381, 20)
(177190, 25)


---

## Création du graph

On récupère les ID unique des projets

In [6]:
project_id = list(project["id"].unique())

Pour chaque projet je récupère les id des organisations qui y participent. Tout ça dans un dictionnaire.

In [7]:
projet_orga = {}
for projet in project_id:
    projet_orga[projet] = list(organization[organization["projectID"] == projet]["organisationID"])

Création d'un graph vide

In [8]:
G = nx.Graph()

Fonction de création du graph (prendre plus d'une heure à faire tourner)

In [9]:
########### /!\ TOURNE INDEFINIEMENT /!\ ##############

yes_counter = 0
for i, tq in zip(range(1, len(projet_orga.keys())), tqdm(range(len(projet_orga.keys())))):
#     print(projet_orga[project_id[i]])
    for t in range(len(projet_orga.keys())):
#         G.add_node(project_id[i - 1])
        G.add_node(project_id[i - 1], label=project_id[i - 1])
        for y in projet_orga[project_id[i - 1]]:
#         print(f"i: {i} | i+1: {projet_orga[project_id[i+1]]} | y: {y}")
            if G.has_edge(project_id[i - 1], project_id[t]) is False:
                for x in projet_orga[project_id[t]]:
            
                    if y == x:
                        yes_counter += 1
                        G.add_edge(project_id[i-1], project_id[t])

G.remove_edges_from(nx.selfloop_edges(G))
# On supprime les boucles du graph car cela n'apporte pas d'information et cela prend de la place

  0%|                                                                              | 8/35381 [00:02<3:31:10,  2.79it/s]


KeyboardInterrupt: 

Pour importer un graph sauvegardé au format .pickle

In [10]:
 G = pickle.load(open("./graphs_save/graphe_3_labeled.pickle", "rb"))

In [11]:
G.number_of_nodes()

35381

In [12]:
G.number_of_edges()

9982209

---

## Détection de communauté

### Détection de communauté

In [13]:
partition = community_louvain.best_partition(G)

In [19]:
print(f"Nombre de communautés : {len(set(partition.values()))}")

Nombre de communautés : 4377


On ajoute les communauté en tant qu'attributs aux noeuds du graph

In [34]:
nx.set_node_attributes(G, partition, 'partition')

### Sauvegarde des communautés

Pour sauvegarder le graph au format .pickle

In [35]:
pickle.dump(G, open('./graphs_save/graphe_3_labeled.pickle', 'wb'))

Pour sauvegarder le graph en format gexf pour l'importer sur Gephi

In [ ]:
nx.write_gexf(G, "./graphs_save/graphe_3_labeled.gexf")

On crée un DataFrame avec en index les id des noeuds et leur communauté

In [14]:
partition_df = pd.DataFrame.from_dict(partition, orient="index").astype({0:"int32"})

On ajoute les communautés au DataFrame de données

In [17]:
project = project.join(partition_df, on="id")
project.rename({0:"Community"}, axis=1, inplace=True)

In [18]:
project.head(2)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,H2020,H2020-LCE-2016-2017,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1


### Modification et analyse des communautés

On calcule la proportion du chaque communauté par rapport à tous le data set 

In [32]:
proportions = project['Community'].value_counts(normalize=True).to_frame()
proportions.rename({"Community": "Proportion"}, inplace=True, axis=1)

In [33]:
proportions

,Proportion
1,0.250276
4,0.094599
16,0.073034
12,0.068596
7,0.063763
...,...
1639,0.000028
1640,0.000028
1641,0.000028
1642,0.000028


On recupère l'index (ici les communautés) pour lesquelles la proportion est inférieure à 1% du data set 

In [46]:
index_label = proportions[proportions["Proportion"] <= 0.01].index.to_list()

On modifie la valeur des communautés pour lesquelles la proportion est inférieure à 1%

In [62]:
def community_filter(x: int):
    if x in index_label:
        return -99
    else:
        return x

On test le code avec une copie du DataFrame (va changer à l'avenir)

In [63]:
project_prepared = project.copy()
project_prepared["Community"] = project["Community"].apply(community_filter)

In [72]:
project_prepared.head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,-99
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,H2020,H2020-LCE-2016-2017,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1
2,784994,PentaHelix,CLOSED,Multi stakeholder and governance approach for ...,2018-03-01,2021-09-30,"1808343,75","1808343,75",H2020-EU.3.3.,EE-09-2016-2017,...,H2020,H2020-EE-2016-2017,H2020-EE-2017-CSA-PPI,CSA,NaN,The PentaHelix project is focusing on developi...,2022-04-11 16:42:12,213566,10.3030/784994,1


In [78]:
project_prepared.shape

(35381, 21)

On crée un dictionnaire avec l'ID des projets et leurs nouvelles communautés pour être ajouté aux attributs des noeuds du graph

In [70]:
new_community = project_prepared.set_index('id')["Community"].to_dict()

In [74]:
new_community

{817296: -99,
 764717: 1,
 784994: 1,
 841546: 1,
 716923: 15,
 814427: 1,
 751782: 3,
 727745: 1,
 740634: 4,
 771082: 1,
 894162: 1,
 678783: 3,
 101032680: 4,
 776740: 1,
 739676: -99,
 843269: 2,
 871647: -99,
 644821: 3,
 727524: 2,
 740829: 1,
 852169: 3,
 101029193: 16,
 882725: -99,
 759207: 16,
 890437: 1,
 101024588: 16,
 699298: 1,
 772705: 16,
 659083: 12,
 681463: -99,
 895260: 8,
 780473: 1,
 661479: 4,
 726904: -99,
 101024021: 3,
 883700: 7,
 794012: 2,
 874434: 1,
 756436: -99,
 101028525: 8,
 803335: 7,
 101007915: 3,
 640277: 1,
 845118: 1,
 876748: -99,
 836139: 7,
 638666: 2,
 802512: 1,
 808490: -99,
 799203: 16,
 704507: 2,
 748114: 7,
 847136: 1,
 810139: 16,
 101002921: 16,
 757247: 9,
 646453: 1,
 787068: 1,
 101026000: 12,
 649829: 1,
 642332: 4,
 704110: 1,
 651993: -99,
 728061: 16,
 842019: 1,
 727028: 1,
 891762: 2,
 832537: 1,
 682472: 16,
 714868: 4,
 753937: 7,
 792489: 3,
 952852: 1,
 957258: 1,
 725798: 9,
 847673: 1,
 661057: -99,
 747046: -99,
 713

In [81]:
print(f"Nombre de communautés après traitement : {len(set(new_community.values()))}")

Nombre de communautés après traitement : 16


On ajoute ces nouvelles communautés au graph ! 

In [75]:
nx.set_node_attributes(G, new_community, 'new_partition')

On enregistre ce nouveau graph

In [76]:
pickle.dump(G, open('./graphs_save/graph_3_labeled_new_partition.pickle', 'wb'))

In [77]:
nx.write_gexf(G, "./graphs_save/graph_3_labeled_new_partition.gexf")